Visualization babysteps. 
We start with the code of Ch 3 of 'Data Science from scratch' by Joel Grus
But we'll work with real gdp data

In [ ]:
# all imports at the top
from matplotlib import pyplot as plt
import ipywidgets as widgets
import pandas as pd
import requests
import zipfile
from io import BytesIO
from pathlib import Path
from IPython.display import clear_output, display


In [1]:
# helper functions
def download_extract_gdp_data(url = 'https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.PP.CD?downloadformat=csv', destination_file : Path = Path('../data/gdp_ppp.csv'))-> None:
    """
    Download and extract GDP data from the World Bank API.
    
    Parameters:
    url (str): The URL to download the data from.
    destination_file (Path): The path to the destination file where it saves the extracted file. Defaults to '../data/gdp_ppp.csv'.
    
    Returns:
    None
    """
    print(f"Downloading data from the World Bank API {url} to {destination_file} ...")
    # download the csv file from the World Bank API
    response = requests.get(url)
    # unzip the contents, find the csv file that starts with API, and save it to the data folder
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        for filename in z.namelist():
            if filename.startswith('API_') and filename.endswith('.csv'):
                with open(destination_file, 'wb') as f:
                    f.write(z.read(filename))
                # save the csv file to the data folder as gdp_ppp.csv
                print(f"Downloaded {filename} to {destination_file}")
                break
    

NameError: name 'Path' is not defined

In [ ]:
# check if data folder has the gdp-ppp csv file
# if not, download it from the World Bank API

data_folder = Path("../data")
data_folder.mkdir(exist_ok=True)
csv_file = data_folder.joinpath("gdp_ppp.csv")
#TODO: look at the lates data on the World Bank API and compare its metadata with the one in the csv file, 
# if they are different, download the new csv file
if not csv_file.exists():
    download_extract_gdp_data(destination_file=csv_file) 
# read the csv file into a pandas dataframe
df = pd.read_csv(csv_file, skiprows=4)
# ask the user for the country name
# get the list of countries from the dataframe
countries = df['Country Name'].unique()
# create a dropdown widget for the country name
country_dropdown = widgets.Dropdown(
    options=countries,
    description='Country:',
    disabled=False,
)

output = widgets.Output()

#create a 1-line table all available gdp data for the selected country
def display_country_data(country_name):
    """
    Display the GDP data for the selected country.
    
    Parameters:
    country_name (str): The name of the country to display data for.
    
    Returns:
    None
    """
    # filter the dataframe for the first row + the row of the  selected country
    country_data = df[df['Country Name'] == country_name]
    country_data = country_data.iloc[0]
    years = df.columns[4:]
    # filter the dataframe for the years columns
    country_data = country_data[years]
    # rename the columns to be the years
    country_data.columns = years
    #drop the missing values
    country_data = country_data.dropna()
    # plot the data as a line chart
    plt.plot(country_data.index, country_data.values)
    plt.title(f"GDP per capita in {country_name}")
    plt.xlabel('Year')
    plt.ylabel('GDP per capita (PPP)')
    plt.xticks(rotation=45)
    plt.grid()
    plt.show()
    
    
    
# a function to update the table when the country is changed
def update_country_data(change):
    """
    Update the GDP data for the selected country.
    
    Parameters:
    change (dict): The change event from the dropdown widget.
    
    Returns:
    None
    """
    with output:
        clear_output(wait=True)
        display_country_data(change['new'])
# bind the update function to the dropdown widget
country_dropdown.observe(update_country_data, names='value')
# display the dropdown widget
display(country_dropdown, output)

In [ ]:
dropdown = widgets.Dropdown(
    options=['France', 'Germany', 'Russia'],
    value='France',
    description='Country:',
)

# Create an output area (safe place for dynamic prints)
output = widgets.Output()

# Update function
def update_selected(change):
    with output:
        clear_output(wait=True)
        print("Selected country:", change['new'])

# Bind and display once
dropdown.observe(update_selected, names='value')
display(dropdown, output)

# Show the initial selection
with output:
    print("Selected country:", dropdown.value)